# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,9.85,71,75,5.14,CA,1690221874
1,1,bethel,41.3712,-73.4140,30.30,56,20,6.17,US,1690222467
2,2,touros,-5.1989,-35.4608,28.27,61,36,8.54,BR,1690221910
3,3,grytviken,-54.2811,-36.5092,-5.71,75,86,4.51,GS,1690222467
4,4,aur,8.1499,171.1687,28.43,75,82,8.12,MH,1690222467


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.75,
    color = "City")

# Display the map
humidity_city

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[city_data_df['Humidity']< 50]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,howard springs,-12.4970,131.0470,16.95,41,7,3.86,AU,1690222469
17,17,kidal,18.4411,1.4078,39.29,21,23,3.46,ML,1690222469
33,33,port hedland,-20.3167,118.5667,12.03,43,0,3.60,AU,1690222472
46,46,yellowknife,62.4560,-114.3525,25.11,34,10,3.77,CA,1690222210
48,48,la union,-5.4000,-80.7500,29.78,44,24,5.87,PE,1690222475


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
14,14,howard springs,-12.4970,131.0470,16.95,41,7,3.86,AU,1690222469,
17,17,kidal,18.4411,1.4078,39.29,21,23,3.46,ML,1690222469,
33,33,port hedland,-20.3167,118.5667,12.03,43,0,3.60,AU,1690222472,
46,46,yellowknife,62.4560,-114.3525,25.11,34,10,3.77,CA,1690222210,
48,48,la union,-5.4000,-80.7500,29.78,44,24,5.87,PE,1690222475,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit":20,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
 
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
howard springs - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
yellowknife - nearest hotel: No hotel found
la union - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
queenstown - nearest hotel: Queens Hotel
as sulayyil - nearest hotel: No hotel found
clovis - nearest hotel: BEST WESTERN Clovis Cole
vallenar - nearest hotel: Hotel del Marqués
vernon - nearest hotel: Okanagan Royal Park Inn
monte patria - nearest hotel: Hotel Monte Cordillera
chongwe - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
lemoore - nearest hotel: Travelodge by Wyndham Lemoore Near Naval Air Station
belle fourche - nearest hotel: Kings Inn
ascension - nearest hotel: Cajun Country Inn
labrador city - nearest hotel: Carol Inn
turpan - nearest hotel: Oriental
torghay - nearest hotel: No hotel found
lodwar - nearest hotel: Lodwar Modern Lodge
itaituba - nearest hotel:

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
14,14,howard springs,-12.4970,131.0470,16.95,41,7,3.86,AU,1690222469,No hotel found
17,17,kidal,18.4411,1.4078,39.29,21,23,3.46,ML,1690222469,No hotel found
33,33,port hedland,-20.3167,118.5667,12.03,43,0,3.60,AU,1690222472,The Esplanade Hotel
46,46,yellowknife,62.4560,-114.3525,25.11,34,10,3.77,CA,1690222210,No hotel found
48,48,la union,-5.4000,-80.7500,29.78,44,24,5.87,PE,1690222475,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
554,554,chiras,35.4167,65.9823,14.88,41,47,0.81,AF,1690222610,No hotel found
556,556,privolzhskiy,51.4000,46.0333,22.39,43,93,2.42,RU,1690222610,Гостиница
558,558,zhosaly,45.4878,64.0781,34.58,21,20,4.99,KZ,1690222610,Gostinitsa Yeseta
559,559,lerma,42.0261,-3.7598,18.31,24,68,4.08,ES,1690222610,DC


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.75,
    color = "City",
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)